# KampInceptionResNet

## Import Libarary

In [ ]:
import torch
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, confusion_matrix, classification_report

from kamp.preprocess import KampDataLoader
from kamp.utils import KampDataset, evaluate_model
from kamp.models import KampInceptionNet

In [8]:
help(KampDataLoader)

Help on class KampDataLoader in module kamp.preprocess:

class KampDataLoader(builtins.object)
 |  KampDataLoader(path, nan_grid={'drop_features': ['line', 'name', 'mold_name', 'time', 'date', 'emergency_stop', 'molten_volume', 'registration_time'], 'simple_fill_dict': {'tryshot_signal': 'No', 'heating_furnace': 'C'}, 'mode_fill_features': ['upper_mold_temp3', 'lower_mold_temp3', 'molten_temp'], 'mode_criterion': 'mold_code'}, do_count_trend=True, drop_count=True, encode_grid={'working': ['정지', '가동'], 'tryshot_signal': ['No', 'D'], 'heating_furnace': ['A', 'B', 'C'], 'mold_code': [8412, 8413, 8573, 8576, 8600, 8722, 8917]}, outlier_method='iso', iso_outlier_rate=0.015, p_threshold=0.05, get_useful_p_data=False, do_resample=True, downsampled_pass_rate=0.5, upsampled_fail_rate_about_pass=1.0, upsample_method='smote', do_pca=False, variance_rate=0.95)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, path, nan_grid={'drop_features': ['line', 'name', 'mold_name', 'time', 'date', 'emer

## Load Data

In [9]:
DATA_PATH = './data/경진대회용 주조 공정최적화 데이터셋.csv'

data_loader = KampDataLoader(
    path = DATA_PATH,

    # 처리 안한게 더 좋았음
    # 처리 안한 것 : 0.944
    # 처리 한 것 : 최대 0.922
    do_count_trend=False,
    drop_count=False,

    get_useful_p_data=True,
    p_threshold=0.05,

    outlier_method='iso',
    iso_outlier_rate=0.0075,

    do_resample=False,
    # downsampled_pass_rate=1.0,
    # upsampled_fail_rate_about_pass=0.25,
    # upsample_method='adasyn',
)

data_loader.process()

data = data_loader.load()

x_train = data['train_data']
y_train = data['train_label']
x_test = data['test_data']
y_test=  data['test_label']

==================== [Data Process Start] ==================== 

[Process Log] Loading Raw Data...
[Process Log] Done

[Process Log] Processing Nan Value...
[Process Log] Done

[Process Log] Encoding Categorical Features...
[Process Log] Done

[Process Log] Removing Outliers (IsoForest)...
[Outlier-Remover Log] With Outliers Shape : (89753, 23)
[Outlier-Remover Log] Without Outliers Shape : (89079, 23)
[Process Log] Done

[Process Log] T-Testing...
[Process Log] Done

[Process Log] Data Scaling (MinMaxScaler)...
[Process Log] Done

[Process Log] Train Test Spliting...
[Process Log] Done

======================= [Done] =======================


## Modeling

In [10]:
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 10
NUM_THREADS = 6

torch.manual_seed(RANDOM_SEED)
torch.set_num_threads(NUM_THREADS)

In [11]:
train_dataset = KampDataset(inputs=x_train, labels=y_train)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [12]:
param_grid = {
    'lr' : 0.1,
    'epochs' : EPOCHS
}

model, history = KampInceptionNet(**param_grid).fit(train_dataloader)

[Epoch 1 | Loss 0.3737] : 100%|██████████| 2227/2227 [03:01<00:00, 12.24it/s]
[Epoch 2 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:05<00:00, 12.00it/s]
[Epoch 3 | Loss 0.3733] : 100%|██████████| 2227/2227 [03:25<00:00, 10.83it/s]
[Epoch 4 | Loss 0.3734] : 100%|██████████| 2227/2227 [04:01<00:00,  9.23it/s]
[Epoch 5 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:08<00:00, 11.79it/s]
[Epoch 6 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:04<00:00, 12.10it/s]
[Epoch 7 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:02<00:00, 12.19it/s]
[Epoch 8 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:01<00:00, 12.26it/s]
[Epoch 9 | Loss 0.3735] : 100%|██████████| 2227/2227 [03:05<00:00, 12.02it/s]
[Epoch 10 | Loss 0.3734] : 100%|██████████| 2227/2227 [03:03<00:00, 12.16it/s]


## Evaluation

In [13]:
test_dataset = KampDataset(inputs=x_test, labels=y_test)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [14]:
true_labels, predicted_labels = evaluate_model(model, test_dataloader)

f1 = f1_score(true_labels, predicted_labels, average='weighted', zero_division=1)
print(f"\nF1 Socre : {f1:.4f}\n")

conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix : ")
print(conf_matrix, '\n')

print("Classification Report : ")
print(classification_report(true_labels, predicted_labels), '\n')

100%|██████████| 557/557 [00:12<00:00, 43.19it/s]


F1 Socre : 0.9451

Confusion Matrix : 
[[17160     0]
 [  656     0]] 

Classification Report : 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     17160
           1       0.00      0.00      0.00       656

    accuracy                           0.96     17816
   macro avg       0.48      0.50      0.49     17816
weighted avg       0.93      0.96      0.95     17816
 




c:\Users\USER\anaconda3\envs\kamp_competition\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\envs\kamp_competition\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\envs\kamp_competition\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali